In [137]:
class Download:
    def __init__(self,base,date,qdate=None):
        import os
        import logging     
        self.date=date
        self.qdate=qdate
        self.base=base
        try:
            datesplit=self.date.split('-')
            self.day=datesplit[0]
            self.month=datesplit[1]
            self.year=datesplit[2]
            self.datevar=f'{self.year}-{self.month}-{self.day}' #get day string in reverse order, so yyyy-mm-dd
        except:
            raise Date_Error('Date has not been inputted correctly, it should be dd-mm-yyyy')
        try:
            self.statpath=f'{self.base}/{self.date}'
            os.mkdir(f'{self.base}/{self.date}')
        except:
            logging.warning(f"Directory '{self.statpath}' might already exist, or cannot be formed")
        if not self.qdate==None:
            try:
                self.quietpath=f'{self.base}/{self.qdate}'
                os.mkdir(f'{self.base}/{self.qdate}')
            except:
                logging.warning(f"Directory '{self.quietpath}' might already exist, or cannot be formed")
        else:
            self.quietpath=None
        
    def download_data(self,day,month,year,station,types=True):
        from urllib.request import urlretrieve
        import os
        import logging
        day=str(day).zfill(2)
        month=str(month).zfill(2)
        logging.info(f'Downloading data for station {station} on {day}-{month}-{year}')
        if types==True: #minute data
            URL="ftp://ftp.seismo.nrcan.gc.ca/intermagnet/minute/definitive/IAGA2002"
            try:
                urlretrieve(f'{URL}/{year}/{month}/{station}{year}{month}{day}dmin.min.gz', f'{self.base}/{day}-{month}-{year}/{station}{year}{month}{day}dmin.min.gz')
            except:
                raise Exception('Data does not exist for given input, station might not be recorded yet. Input should have length: 3-2-2-4')
            os.system(f'gunzip {self.base}/{day}-{month}-{year}/{station}{year}{month}{day}dmin.min.gz')
            
        else: #second data
            URL="ftp://ftp.seismo.nrcan.gc.ca/intermagnet/second/quasi-definitive/IAGA2002"
            try:
                urlretrieve(f'{URL}/{year}/{month}/{station}{year}{month}{day}qsec.sec.gz', f'{self.base}/{day}-{month}-{year}/{station}{year}{month}{day}qsec.sec.gz')
            except:
                raise Exception('Data does not exist for given input, station might not be recorded yet. Input should have length: 3-2-2-4')
            os.system(f'gunzip {self.base}/{day}-{month}-{year}/{station}{year}{month}{day}qsec.sec.gz')
            
    def standard_download(self,types=True): #download data automatic from intermagnet
        import logging
        import os
        list_of_stations=['fur','had','bfe','clf','dou','esk','ler','ngk','ups','wng']
        for station in list_of_stations:
            try:
                self.download_data(self.day,self.month,self.year,station,types)
            except:
                logging.warning(f'Data could not be downloaded for station {station}')
        if self.qdate==None:
            qday, qmonth, qyear = self.find_quiet_date()
            qday=str(qday).zfill(2)
            qmonth=str(qmonth).zfill(2)
            self.qdate=f'{qday}-{qmonth}-{qyear}'
            logging.info(f'Quiet day is {self.qdate}')
            print(f'Quiet day is {self.qdate}')
            try:
                self.quietpath=f'{self.base}/{self.qdate}'
                os.mkdir(f'{self.base}/{self.qdate}')
                for station in list_of_stations:
                    try:
                        self.download_data(qday,qmonth,qyear,station,types)
                    except:
                        logging.warning(f'Data could not be downloaded for station {station}')
            except:
                logging.warning(f"Directory '{self.quietpath}' might already exist, or cannot be formed")
        
    def find_quiet_date(self):
        from urllib.request import urlretrieve
        import os
        import logging
        import datetime
        URL='ftp://ftp.gfz-potsdam.de/pub/home/obs/kp-ap/quietdst'
        monthlist=[0,2,3,4,5,6,7,9,10,11,12,13,14]
        try: #retrieve data from the Potzdam website
            urlretrieve(f'{URL}/qd{self.year[0:3]}0{self.year[2]}9.txt', f'{self.base}/Kp_index_{self.year}.txt')
        except:
            try:
                urlretrieve(f'{URL}/qd{self.year[0:3]}0{self.year[2]}x.txt', f'{self.base}/Kp_index_{self.year}.txt')
            except:
                raise Exception('URL could not be retrieved, check your date string!')
        # find correct files and extract quiet days of the month
        if self.month=='12' and self.year[3]=='9':
            newyear=str(int(self.year)+1)
            try:
                urlretrieve(f'{URL}/qd{newyear[0:3]}0{newyear[2]}9.txt', f'{self.base}/Kp_index_{newyear}.txt')
            except:
                try:
                    urlretrieve(f'{URL}/qd{newyear[0:3]}0{newyear[2]}x.txt', f'{self.base}/Kp_index_{newyear}.txt')
                except:
                    raise Exception('URL could not be retrieved, check your date string!')
            f=open(f'{self.base}/Kp_index_{self.year}.txt')
            for counter,line in enumerate(f):
                if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)-1]+2:
                    words=line.split()
                    word=words[2]
                    option0=[''.join(i for i in words[2] if i.isdigit()), int(self.month)-1, self.year]
                    option0A=[''.join(i for i in words[3] if i.isdigit()), int(self.month)-1, self.year]

                if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)]+2:
                    words=line.split()
                    word=words[2]
                    option1=[''.join(i for i in words[2] if i.isdigit()), self.month, self.year]
                    option1A=[''.join(i for i in words[3] if i.isdigit()), self.month, self.year]
                    f.close()
                    os.system(f'rm {self.base}/Kp_index_{self.year}.txt')
                    break
            f=open(f'{self.base}/Kp_index_{newyear}.txt')
            for counter,line in enumerate(f):
                if counter==4:
                    words=line.split()
                    word=words[2]
                    option2=[''.join(i for i in words[2] if i.isdigit()), 1, int(self.year)+1]
                    option2A=[''.join(i for i in words[3] if i.isdigit()), 1, int(self.year)+1]
                    f.close()
                    os.system(f'rm {self.base}/Kp_index_{newyear}.txt')
                    break
            
        elif self.month=='01' and self.year[3]=='0':
            newyear=str(int(self.year)-1)
            try:
                urlretrieve(f'{URL}/qd{newyear[0:3]}0{newyear[2]}9.txt', f'{self.base}/Kp_index_{newyear}.txt')
            except:
                try:
                    urlretrieve(f'{URL}/qd{newyear[0:3]}0{newyear[2]}x.txt', f'{self.base}/Kp_index_{newyear}.txt')
                except:
                    raise Exception('URL could not be retrieved, check your date string!')
            f=open(f'{self.base}/Kp_index_{newyear}.txt')
            for counter,line in enumerate(f):
                if counter==130:
                    words=line.split()
                    word=words[2]
                    option0=[''.join(i for i in words[2] if i.isdigit()), 12, int(self.year)-1]
                    option0A=[''.join(i for i in words[3] if i.isdigit()), 12, int(self.year)-1]
                    f.close()
                    os.system(f'rm {self.base}/Kp_index_{newyear}.txt')
                    break
            f=open(f'{self.base}/Kp_index_{self.year}.txt')
            for counter,line in enumerate(f):
                if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)]+2:
                    words=line.split()
                    word=words[2]
                    option1=[''.join(i for i in words[2] if i.isdigit()), self.month, self.year]
                    option1A=[''.join(i for i in words[3] if i.isdigit()), self.month, self.year]

                if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)+1]+2:
                    words=line.split()
                    word=words[2]
                    option2=[''.join(i for i in words[2] if i.isdigit()), int(self.month)+1, self.year]
                    option2A=[''.join(i for i in words[3] if i.isdigit()), int(self.month)+1, self.year]
                    f.close()
                    os.system(f'rm {self.base}/Kp_index_{self.year}.txt')
                    break
        
        else:
            f=open(f'{self.base}/Kp_index_{self.year}.txt')
            for counter,line in enumerate(f):
                if self.month=='12':
                    if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)-1]+2:
                        words=line.split()
                        word=words[2]
                        option0=[''.join(i for i in words[2] if i.isdigit()), int(self.month)-1, self.year]
                        option0A=[''.join(i for i in words[3] if i.isdigit()), int(self.month)-1, self.year]
                
                    if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)]+2:
                        words=line.split()
                        word=words[2]
                        option1=[''.join(i for i in words[2] if i.isdigit()), self.month, self.year]
                        option1A=[''.join(i for i in words[3] if i.isdigit()), self.month, self.year]
                        
                    if counter==(int(self.year)-int(self.year[0:3])*10+1)*14+4:
                        words=line.split()
                        word=words[2]
                        option2=[''.join(i for i in words[2] if i.isdigit()), 1, int(self.year)+1]
                        option2A=[''.join(i for i in words[3] if i.isdigit()), 1, int(self.year)+1]
                        f.close()
                        os.system(f'rm {self.base}/Kp_index_{self.year}.txt')
                        break
                elif self.month=='1':
                    if counter==(int(self.year)-int(self.year[0:3])*10)*14+2:
                        words=line.split()
                        word=words[2]
                        option0=[''.join(i for i in words[2] if i.isdigit()), 12, int(self.year)-1]
                        option0A=[''.join(i for i in words[3] if i.isdigit()), 12, int(self.year)-1]
                
                    if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)]+2:
                        words=line.split()
                        word=words[2]
                        option1=[''.join(i for i in words[2] if i.isdigit()), self.month, self.year]
                        option1A=[''.join(i for i in words[3] if i.isdigit()), self.month, self.year]
                        
                    if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)+1]+2:
                        words=line.split()
                        word=words[2]
                        option2=[''.join(i for i in words[2] if i.isdigit()), int(self.month)+1, self.year]
                        option2A=[''.join(i for i in words[3] if i.isdigit()), int(self.month)+1, self.year]
                        f.close()
                        os.system(f'rm {self.base}/Kp_index_{self.year}.txt')
                        break
                else:
                    if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)-1]+2:
                        words=line.split()
                        word=words[2]
                        option0=[''.join(i for i in words[2] if i.isdigit()), int(self.month)-1, self.year]
                        option0A=[''.join(i for i in words[3] if i.isdigit()), int(self.month)-1, self.year]
                
                    if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)]+2:
                        words=line.split()
                        word=words[2]
                        option1=[''.join(i for i in words[2] if i.isdigit()), self.month, self.year]
                        option1A=[''.join(i for i in words[3] if i.isdigit()), self.month, self.year]
                        
                    if counter==(int(self.year)-int(self.year[0:3])*10)*14+monthlist[int(self.month)+1]+2:
                        words=line.split()
                        word=words[2]
                        option2=[''.join(i for i in words[2] if i.isdigit()), int(self.month)+1, self.year]
                        option2A=[''.join(i for i in words[3] if i.isdigit()), int(self.month)+1, self.year]
                        f.close()
                        os.system(f'rm {self.base}/Kp_index_{self.year}.txt')
                        break
                        
        #which option is closest to disturbed day?
        logging.info(f"optional quiet days are (previous/this/next month): {option0} OR {option0A} / {option1} OR {option1A} / {option2} OR {option2A}")
        datestring=[datetime.datetime(int(option0[2]),int(option0[1]),int(option0[0])),datetime.datetime(int(option1[2]),int(option1[1]),int(option1[0])),datetime.datetime(int(option2[2]),int(option2[1]),int(option2[0])),datetime.datetime(int(option0A[2]),int(option0A[1]),int(option0A[0])),datetime.datetime(int(option1A[2]),int(option1A[1]),int(option1A[0])),datetime.datetime(int(option2A[2]),int(option2A[1]),int(option2A[0]))]
        Quiet=min(datestring, key=lambda x: abs(x - datetime.datetime(int(self.year),int(self.month),int(self.day))))
        return Quiet.day, Quiet.month, Quiet.year

In [138]:
testcase=Download('/usr/people/out/Documents/Magnetic_field/magnetic_data','20-11-2003')
testcase.standard_download()

Quiet day is 27-11-2003
